In [1]:
import time
import numpy as np
import psi4

In [2]:
class helper_diis(object):
    def __init__(self, t1, t2, max_diis):

        self.oldt1 = t1.copy()
        self.oldt2 = t2.copy()
        self.diis_vals_t1 = [t1.copy()]
        self.diis_vals_t2 = [t2.copy()]
        self.diis_errors = []
        self.diis_size = 0
        self.max_diis = max_diis

    def add_error_vector(self, t1, t2):

        # Add DIIS vectors
        self.diis_vals_t1.append(t1.copy())
        self.diis_vals_t2.append(t2.copy())
        # Add new error vectors
        error_t1 = (self.diis_vals_t1[-1] - self.oldt1).ravel()
        error_t2 = (self.diis_vals_t2[-1] - self.oldt2).ravel()
        self.diis_errors.append(np.concatenate((error_t1, error_t2)))
        self.oldt1 = t1.copy()
        self.oldt2 = t2.copy()

    def extrapolate(self, t1, t2):

        # Limit size of DIIS vector
        if (len(self.diis_vals_t1) > self.max_diis):
            del self.diis_vals_t1[0]
            del self.diis_vals_t2[0]
            del self.diis_errors[0]

        self.diis_size = len(self.diis_vals_t1) - 1

        # Build error matrix B
        B = np.ones((self.diis_size + 1, self.diis_size + 1)) * -1
        B[-1, -1] = 0

        for n1, e1 in enumerate(self.diis_errors):
            B[n1, n1] = np.dot(e1, e1)
            for n2, e2 in enumerate(self.diis_errors):
                if n1 >= n2: continue
                B[n1, n2] = np.dot(e1, e2)
                B[n2, n1] = B[n1, n2]

        B[:-1, :-1] /= np.abs(B[:-1, :-1]).max()

        # Build residual vector
        resid = np.zeros(self.diis_size + 1)
        resid[-1] = -1

        # Solve pulay equations
        ci = np.linalg.solve(B, resid)

        # Calculate new amplitudes
        t1 = np.zeros_like(self.oldt1)
        t2 = np.zeros_like(self.oldt2)
        for num in range(self.diis_size):
            t1 += ci[num] * self.diis_vals_t1[num + 1]
            t2 += ci[num] * self.diis_vals_t2[num + 1]

        # Save extrapolated amplitudes to old_t amplitudes
        self.oldt1 = t1.copy()
        self.oldt2 = t2.copy()

        return t1, t2


In [55]:
class HelperCCEnergy(object):
    def __init__(self, mol, rhf_e, rhf_wfn, memory=2):

        print("\nInitalizing CCSD object...\n")

        # Integral generation from Psi4's MintsHelper
        time_init = time.time()

        self.rhf_e = rhf_e
        self.wfn = rhf_wfn

        self.ccsd_corr_e = 0.0
        self.ccsd_e = 0.0

        self.ndocc = self.wfn.doccpi()[0]
        self.nmo = self.wfn.nmo()
        self.memory = memory
        self.C = self.wfn.Ca()
        self.npC = np.asarray(self.C)

        self.mints = psi4.core.MintsHelper(self.wfn.basisset())
        H = np.asarray(self.mints.ao_kinetic()) + np.asarray(
            self.mints.ao_potential())
        self.nmo = H.shape[0]

        # Update H, transform to MO basis
        H = np.einsum('uj,vi,uv', self.npC, self.npC, H)

        print('Starting AO ->  MO transformation...')

        ERI_Size = self.nmo * 128.e-9
        memory_footprint = ERI_Size * 5
        if memory_footprint > self.memory:
            psi.clean()
            raise Exception(
                "Estimated memory utilization (%4.2f GB) exceeds numpy_memory \
                            limit of %4.2f GB." % (memory_footprint,
                                                   self.memory))

        # Integral generation from Psi4's MintsHelper
        self.MO = np.asarray(self.mints.mo_eri(self.C, self.C, self.C, self.C))
        # Physicist notation
        self.MO = self.MO.swapaxes(1,2)
        print("Size of the ERI tensor is %4.2f GB, %d basis functions." %
              (ERI_Size, self.nmo))

        # Update nocc and nvirt
        self.nocc = self.ndocc
        self.nvirt = self.nmo - self.nocc

        # Make slices
        self.slice_o = slice(0, self.nocc)
        self.slice_v = slice(self.nocc, self.nmo)
        self.slice_a = slice(0, self.nmo)
        self.slice_dict = {
            'o': self.slice_o,
            'v': self.slice_v,
            'a': self.slice_a
        }

        # Compute Fock matrix
        self.F = H + 2.0 * np.einsum('pmqm->pq',
                                     self.MO[:, self.slice_o, :, self.slice_o])
        self.F -= np.einsum('pmmq->pq',
                            self.MO[:, self.slice_o, self.slice_o, :])

        ### Occupied and Virtual orbital energies
        Focc = np.diag(self.F)[self.slice_o]
        Fvir = np.diag(self.F)[self.slice_v]

        self.Dia = Focc.reshape(-1, 1) - Fvir
        self.Dijab = Focc.reshape(-1, 1, 1, 1) + Focc.reshape(
            -1, 1, 1) - Fvir.reshape(-1, 1) - Fvir

        ### Construct initial guess
        print('Building initial guess...')
        # t^a_i
        self.t1 = np.zeros((self.nocc, self.nvirt))
        # t^{ab}_{ij}
        self.t2 = self.MO[self.slice_o, self.slice_o, self.slice_v,
                          self.slice_v] / self.Dijab

        print('\n..initialized CCSD in %.3f seconds.\n' %
              (time.time() - time_init))

    # occ orbitals  : i, j, k, l, m, n
    # virt orbitals : a, b, c, d, e, f
    # all oribitals : p, q, r, s, t, u, v

    def get_F(self, string):
        if len(string) != 2:
            psi4.core.clean()
            raise Exception('get_F: string %s must have 2 elements.' % string)
        return self.F[self.slice_dict[string[0]], self.slice_dict[string[1]]]
    
    def get_MO(self, string):
        if len(string) != 4:
            psi4.core.clean()
            raise Exception('get_MO: string %s must have 4 elements.' % string)
        return self.MO[self.slice_dict[string[0]], self.slice_dict[string[1]],
                       self.slice_dict[string[2]], self.slice_dict[string[3]]]

    def build_restricted_intermediates(self):
        asym_t2 = 2*self.t2 - self.t2.swapaxes(0,1)
    
        c_ijab = np.einsum('ia,jb->ijab',self.t1,self.t1)
        c_ijab += self.t2
    
        I_ai = 2*np.einsum('aeim,me->ai',self.get_MO('vvoo'),self.t1)
        I_ai -= np.einsum('eaim,me->ai',self.get_MO('vvoo'),self.t1)
        I_ai += self.get_F('vo')
    
        I_ba = 2*np.einsum('beam,me->ba',self.get_MO('vvvo'),self.t1)
        I_ba -= np.einsum('bema,me->ba',self.get_MO('vvov'),self.t1)
        I_ba -= 2*np.einsum('ebmn,mnea->ba',self.get_MO('vvoo'),c_ijab)
        I_ba += np.einsum('bemn,mnea->ba',self.get_MO('vvoo'),c_ijab)
        I_ba += self.get_F('vv')-np.diag(np.diag(self.get_F('vv')))
        I_ba -= np.einsum('ma,bm->ba',self.t1,self.get_F('vo'))
    
        I_ji_p = 2*np.einsum('jeim,me->ji',self.get_MO('ovoo'),self.t1)
        I_ji_p -= np.einsum('ejim,me->ji',self.get_MO('vooo'),self.t1)
        I_ji_p += np.einsum('efmi,mjef->ji',self.get_MO('vvoo'),self.t2)
        I_ji_p -= np.einsum('efim,mjef->ji',self.get_MO('vvoo'),self.t2)
        I_ji_p += self.get_F('oo') - np.diag(np.diag(self.get_F('oo')))
        
        I_ji = np.einsum('ei,je->ji',I_ai,self.t1)
        I_ji += I_ji_p
    
        I_klij = np.einsum('efij,klef',self.get_MO('vvoo'),c_ijab)
        tmp = np.einsum('ke,elij->klij',self.t1,self.get_MO('vooo'))
        I_klij += tmp
        I_klij += tmp.swapaxes(0,1).swapaxes(2,3)
        I_klij += self.get_MO('oooo')
      
        I_jbia = -0.5*np.einsum('ebim,jmea->jbia',self.get_MO('vvoo'),c_ijab)
        I_jbia -= np.einsum('jbim,ma->jbia',self.get_MO('ovoo'),self.t1)
        I_jbia += np.einsum('ebia,je->jbia',self.get_MO('vvov'),self.t1)
        I_jbia += self.get_MO('ovov')
        
        I_bjia = np.einsum('beim,mjea->bjia',self.get_MO('vvoo'),self.t2)
        I_bjia -= 0.5*np.einsum('beim,mjae->bjia',self.get_MO('vvoo'),c_ijab)
        I_bjia -= 0.5*np.einsum('mbie,jmae->bjia',self.get_MO('ovov'),self.t2)
        I_bjia += np.einsum('beia,je->bjia',self.get_MO('vvov'),self.t1)
        I_bjia -= np.einsum('bjim,ma->bjia',self.get_MO('vooo'),self.t1)
        I_bjia += self.get_MO('voov')
        
        I_ciab_p = -np.einsum('ciam,mb->ciab',self.get_MO('vovo'),self.t1)
        I_ciab_p -= np.einsum('ma,cimb->ciab',self.t1,self.get_MO('voov'))
        I_ciab_p += self.get_MO('vovv')
        
        x_bjia = np.einsum('beia,je->bjia',self.get_MO('vvov'),self.t1)
        
        I_jkia_p = np.einsum('efia,jkef->jkia',self.get_MO('vvov'),self.t2)
        I_jkia_p += np.einsum('je,ekia->jkia',self.t1,x_bjia)
        I_jkia_p += self.get_MO('ooov')
        
        return asym_t2, c_ijab, I_ai, I_ba, I_ji_p, I_ji, I_klij, I_jbia, I_bjia, I_ciab_p, I_jkia_p
        
    def update_amplitudes_restricted(self):
        asym_t2, c_ijab, I_ai, I_ba, I_ji_p, I_ji, I_klij, I_jbia, I_bjia, I_ciab_p, I_jkia_p = self.build_restricted_intermediates()
        r_t1 = np.einsum('ea,ie->ia',I_ba,self.t1)
        r_t1 -= np.einsum('im,ma->ia',I_ji_p,self.t1)
        r_t1 += np.einsum('em,miea->ia',I_ai,asym_t2)
        r_t1 += 2.0*np.einsum('eima,me->ia',self.get_MO('voov'),self.t1)
        r_t1 -= np.einsum('eiam,me->ia',self.get_MO('vovo'),self.t1)
        r_t1 -= np.einsum('eimn,mnea->ia',self.get_MO('vooo'),asym_t2)
        r_t1 += np.einsum('efma,mief->ia',self.get_MO('vvov'),asym_t2)
        r_t1 += self.get_F('ov')
        
        tmp_t2 = np.einsum('ijae,eb->ijab',self.t2,I_ba)
        tmp_t2 -= np.einsum('imab,jm->ijab',self.t2,I_ji)
        tmp_t2 += 0.5*np.einsum('efab,ijef->ijab',self.get_MO('vvvv'),c_ijab)
        tmp_t2 += 0.5*np.einsum('mnab,ijmn->ijab',c_ijab,I_klij)
        tmp_t2 -= np.einsum('mjae,iemb->ijab',self.t2,I_jbia)
        tmp_t2 -= np.einsum('iema,mjeb->ijab',I_jbia,self.t2)
        tmp_t2 += np.einsum('miea,ejmb->ijab',asym_t2,I_bjia)
        tmp_t2 += np.einsum('ie,ejab->ijab',self.t1,I_ciab_p)
        tmp_t2 -= np.einsum('ma,ijmb->ijab',self.t1,I_jkia_p)
        
        #tmp_t2 += np.einsum('jibe,ea->ijab',self.t2,I_ba)
        #tmp_t2 -= np.einsum('jmba,im->ijab',self.t2,I_ji)
        #tmp_t2 += 0.5*np.einsum('efba,jief->ijab',self.get_MO('vvvv'),c_ijab)
        #tmp_t2 += 0.5*np.einsum('mnba,jimn->ijab',c_ijab,I_klij)
        #tmp_t2 -= np.einsum('mibe,jema->ijab',self.t2,I_jbia)
        #tmp_t2 -= np.einsum('jemb,miea->ijab',I_jbia,self.t2)
        #tmp_t2 += np.einsum('mjeb,eima->ijab',asym_t2,I_bjia)
        #tmp_t2 += np.einsum('je,eiba->ijab',self.t1,I_ciab_p)
        #tmp_t2 -= np.einsum('mb,jima->ijab',self.t1,I_jkia_p)
        
        r_t2 = tmp_t2.copy()
        r_t2 += tmp_t2.swapaxes(0,1).swapaxes(2,3)
        r_t2 += self.get_MO('oovv')
        
        self.t1 = r_t1/self.Dia
        self.t2 = r_t2/self.Dijab
        
    def compute_corr_energy(self):
        c_ijab = self.t2.copy()
        c_ijab += np.einsum('ia,jb->ijab',self.t1,self.t1)
        CCSDcorr_E = 2.0 * np.einsum('abij,ijab->',self.get_MO('vvoo'),c_ijab)
        CCSDcorr_E -= 1.0 * np.einsum('baij,ijab->',self.get_MO('vvoo'),c_ijab)

        self.ccsd_corr_e = CCSDcorr_E
        self.ccsd_e = self.rhf_e + self.ccsd_corr_e
        return CCSDcorr_E

    def compute_energy(self,
                       e_conv=1e-6,
                       r_conv=1e-4,
                       maxiter=100,
                       max_diis=9,
                       start_diis=1):

        ### Start Iterations
        ccsd_tstart = time.time()

        # Compute MP2 energy
        CCSDcorr_E_old = self.compute_corr_energy()
        print(
            "CCSD Iteration %3d: CCSD correlation = %.15f   dE = % .5E   MP2" %
            (0, CCSDcorr_E_old, -CCSDcorr_E_old))

        # Set up DIIS before iterations begin
        diis_object = helper_diis(self.t1, self.t2, max_diis)

        # Iterate!
        for CCSD_iter in range(1, maxiter + 1):

            self.update_amplitudes_restricted()

            # Compute CCSD correlation energy
            CCSDcorr_E = self.compute_corr_energy()

            # Print CCSD iteration information
            print(
                'CCSD Iteration %3d: CCSD correlation = %.15f   dE = % .5E    DIIS = %d'
                % (CCSD_iter, CCSDcorr_E, (CCSDcorr_E - CCSDcorr_E_old),
                   diis_object.diis_size))

            # Check convergence
            if (abs(CCSDcorr_E - CCSDcorr_E_old) < e_conv):
                print('\nCCSD has converged in %.3f seconds!' %
                      (time.time() - ccsd_tstart))
                return CCSDcorr_E

            # Update old energy
            CCSDcorr_E_old = CCSDcorr_E

            #  Add the new error vector
            diis_object.add_error_vector(self.t1, self.t2)

            if CCSD_iter >= start_diis:
                self.t1, self.t2 = diis_object.extrapolate(self.t1, self.t2)


# End HelperCCEnergy class

In [56]:
np.set_printoptions(precision=5, linewidth=200, threshold=200, suppress=True)

# Psi4 setup
psi4.set_memory('1 GB')
psi4.core.set_output_file('output.dat', False)
mol = psi4.geometry("""
O
H 1 1.6
H 1 1.6 2 104.45
symmetry c1
""")

e_tol = 1.0e-6
max_iter = 80

# roots per irrep must be set to do the eom calculation with psi4
psi4.set_options({'basis': 'def2-svp'})

# Compute CCSD energy for required integrals and T-amplitudes
rhf_e, rhf_wfn = psi4.energy('SCF', return_wfn=True)
ccsd = HelperCCEnergy(mol, rhf_e, rhf_wfn)
ccsd.compute_energy()

ccsd_cor_e = ccsd.ccsd_corr_e
ccsd_tot_e = ccsd.ccsd_e
print("\n")
print("{}{}".format('Final CCSD correlation energy:'.ljust(30, '.'),
                    "{:16.15f}".format(ccsd_cor_e).rjust(20, '.')))
print("{}{}".format('Total CCSD energy:'.ljust(30, '.'),
                    "{:16.15f}".format(ccsd_tot_e).rjust(20, ',')))


Initalizing CCSD object...

Starting AO ->  MO transformation...
Size of the ERI tensor is 0.00 GB, 24 basis functions.
Building initial guess...

..initialized CCSD in 0.030 seconds.

CCSD Iteration   0: CCSD correlation = -0.261621350865743   dE =  2.61621E-01   MP2
CCSD Iteration   1: CCSD correlation = -0.249956890123713   dE =  1.16645E-02    DIIS = 0
CCSD Iteration   2: CCSD correlation = -0.272899329469904   dE = -2.29424E-02    DIIS = 1
CCSD Iteration   3: CCSD correlation = -0.275291677542216   dE = -2.39235E-03    DIIS = 2
CCSD Iteration   4: CCSD correlation = -0.287685045817340   dE = -1.23934E-02    DIIS = 3
CCSD Iteration   5: CCSD correlation = -0.288517360873578   dE = -8.32315E-04    DIIS = 4
CCSD Iteration   6: CCSD correlation = -0.289739845459457   dE = -1.22248E-03    DIIS = 5
CCSD Iteration   7: CCSD correlation = -0.291040631184583   dE = -1.30079E-03    DIIS = 6
CCSD Iteration   8: CCSD correlation = -0.291528544224846   dE = -4.87913E-04    DIIS = 7
CCSD Itera

In [52]:
ccsd.get_F('ov')

array([[ 0.     ,  0.     , -0.     , -0.     ,  0.00001, -0.     ,  0.     ,  0.     , -0.     , -0.     ,  0.     , -0.     ,  0.     ,  0.00001,  0.     , -0.     ,  0.     , -0.00001, -0.     ],
       [-0.00001, -0.     , -0.00001,  0.     , -0.00002, -0.     , -0.00002, -0.     ,  0.     , -0.     , -0.     , -0.     ,  0.     ,  0.00006,  0.     ,  0.00001, -0.     ,  0.00007,  0.     ],
       [ 0.     , -0.     ,  0.     , -0.     ,  0.     ,  0.00005, -0.     , -0.     ,  0.     , -0.     ,  0.     ,  0.00002, -0.     , -0.     , -0.00002, -0.     , -0.     ,  0.     ,  0.     ],
       [ 0.     , -0.00002,  0.     ,  0.00003, -0.     , -0.     , -0.     , -0.00001, -0.     , -0.00004, -0.     ,  0.     ,  0.00001, -0.     ,  0.     , -0.     , -0.     , -0.     ,  0.00001],
       [ 0.00001, -0.     , -0.     ,  0.     ,  0.     ,  0.     ,  0.00003,  0.     , -0.     , -0.     ,  0.00001,  0.     ,  0.     , -0.00001,  0.     ,  0.00002, -0.     ,  0.     ,  0.     ]])